# Вариант 1

В файле `sex_bmi_smokers.csv` приведены данные (пол, ИМТ, курит/не курит) о более 1000 испытуемых.

# Задание 1

С помощью критерия согласия Пирсона хи-квадрат проверить согласованность индекса массы тела с нормальным законом (формализовать  основные и альтернативные гипотезы, **реализовать самостоятельно**). Ту же самую задачу решить с помощью другого критерия (тоже формализовать гипотезы, но здесь можно воспользоваться готовой *реализацией*).

## Считывание данных

In [ ]:
import pandas as pd
import numpy as np

import scipy.stats
from scipy.stats import chi2, normaltest, chisquare, norm
from scipy.stats import fisher_exact

In [ ]:
!wget -O /content/sex_bmi_smokers.csv 'https://drive.google.com/uc?id=1gzPRqj7gZetjsipo3xpogYGL76enZDNO'

--2024-05-27 10:55:16--  https://drive.google.com/uc?id=1gzPRqj7gZetjsipo3xpogYGL76enZDNO
Resolving drive.google.com (drive.google.com)... 172.217.204.102, 172.217.204.139, 172.217.204.138, ...
Connecting to drive.google.com (drive.google.com)|172.217.204.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1gzPRqj7gZetjsipo3xpogYGL76enZDNO [following]
--2024-05-27 10:55:16--  https://drive.usercontent.google.com/download?id=1gzPRqj7gZetjsipo3xpogYGL76enZDNO
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.26.132, 2607:f8b0:400c:c04::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.26.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26902 (26K) [application/octet-stream]
Saving to: ‘/content/sex_bmi_smokers.csv’

/content/sex_bmi_sm 100%[===================>]  26.27K  --.-KB/s    in 0s      

2024-05-27 10:55:1

In [ ]:
data = pd.read_csv('/content/sex_bmi_smokers.csv')

In [ ]:
data.sample(5)

,sex,bmi,smoker
594,male,40.260,no
457,female,30.495,no
1333,male,30.970,no
1096,female,34.960,yes
454,male,46.530,no


In [ ]:
data.nunique()

sex         2
bmi       548
smoker      2
dtype: int64

## Решение задачи

Формализуем гипотезы

*   нулевая гипотеза $H_0$ - ИМТ согласован с нормальным законом
*   альтернативная гипотеза $H_1$ - ИМТ не согласован с нормальным законом.



### Критерий согласия Пирсона хи-квадрат

In [ ]:
bmi_data = np.array(data['bmi'])
edges = np.linspace(bmi_data.min(), bmi_data.max(), 6)

binned_data = np.histogram(bmi_data, bins=edges)
observed_freq = binned_data[0]

print(edges)
print(binned_data[0])

[15.96  23.394 30.828 38.262 45.696 53.13 ]
[157 553 483 128  17]


In [ ]:
bmi_data = np.array(data['bmi'])
edges = [0, 18.5, 24.9, 29.9, float('inf')]

binned_data = np.histogram(bmi_data, bins=edges)
observed_freq = binned_data[0]

print(edges)
print(binned_data[0])

[0, 18.5, 24.9, 29.9, inf]
[ 20 222 377 719]


In [ ]:
mean = np.mean(bmi_data)
std = np.std(bmi_data)
mean, std

(30.66339686098655, 6.0959076415894256)

In [ ]:
expected_freq = np.array([norm.cdf(edges[i+1], loc=mean, scale=std) - norm.cdf(edges[i], loc=mean, scale=std)
for i in range(len(edges)-1)]) * len(bmi_data)

In [ ]:
observed_freq

array([ 20, 222, 377, 719])

In [ ]:
scipy.stats.chisquare(f_obs=observed_freq, f_exp=expected_freq)

ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
2.4503413706719967e-07

In [ ]:
stat, p = scipy.stats.normaltest(data['bmi'])
print('Statistics=%.3f, p-value=%.5f' % (stat, p))

### Критерий согласия Колмогорова-Смирнова

In [ ]:
stat, p = scipy.stats.kstest(data['bmi'], 'norm', args = (mean, std))
print('Statistics=%.3f, p-value=%.5f' % (stat, p))

Statistics=0.026, p-value=0.31454


Вывод: нулевая гипотеза отвергнута(?).

жалко её(

# Задание 2

С помощью критерия однородности хи-квадрат проверить однородность индекса массы тела курящих и некурящих (формализовать $H_0$ и $H_1$, **реализовать самостоятельно**). Ту же самую задачу решить с помощью другого критерия (тоже формализовать гипотезы, но здесь можно воспользоваться готовой *реализацией*).

## Решение задачи

Формализуем гипотезы

*   нулевая гипотеза $H_0$ - ИМТ курящих и некурящих однородный (у них одинаковое распределение)
*   альтернативная гипотеза $H_1$ - ИМТ курящих и некурящих неоднородный (их распределения отличаются).


### Критерий однородности хи-квадрат

In [ ]:
bins = [0, 18.5, 24.9, 29.9, float('inf')]
labels = ['Underweight', 'Normal', 'Overweight', 'Obese']
data['bmi_category'] = pd.cut(data['bmi'], bins=bins, labels=labels)
data.drop(columns=['bmi'], inplace=True)

In [ ]:
contingency_table = create_contingency_table(data, 'sex', 'bmi_category')

In [ ]:
totals = count_totals(contingency_table)
chi2_value = count_chi2(totals, contingency_table)

print(f"Chi2: {chi2_value}")

Chi2: 2.8830932416639667


In [ ]:
p_value = count_p_value(totals, chi2_value)

print(f"p-value: {p_value}")

p-value: 0.4100033122040875


Гипотеза не опровергнута.

### Критерий однородности Колмогорова-Смирнова

In [ ]:
data = pd.read_csv('/content/sex_bmi_smokers.csv')

In [ ]:
stat, p = scipy.stats.kstest(data[data['smoker'] == 'no']['bmi'], data[data['smoker'] == 'yes']['bmi'])
print('Statistics=%.3f, p-value=%.5f' % (stat, p))

Statistics=0.051, p-value=0.61057


Гипотеза не отвергнута.

# Задание 3

С помощью критерия независимости хи-квадрат проверить независимость индекса массы тела и пола (формализовать $H_0$ и $H_1$, **реализовать самостоятельно**). Ту же самую задачу решить с помощью другого критерия (тоже формализовать гипотезы, но здесь можно воспользоваться готовой *реализацией*).

## Решение

Формализуем гипотезы

*   нулевая гипотеза $H_0$ - ИМТ и пол независимы
*   альтернативная гипотеза $H_1$ - между ИМТ и полом существует зависимость

### Критерий независимости хи-квадрат

In [ ]:
# считаем данные заново на случай, если они были преобразованы в других заданиях
data = pd.read_csv('/content/sex_bmi_smokers.csv')

In [ ]:
data.nunique()

Значений BMI очень много, у нас будет слишком большая матрица (наверное, в этом нет необходимости). Поделим BMI на категории поменьше.

#### Решение с категоризацией BMI

In [ ]:
bins = [0, 18.5, 24.9, 29.9, float('inf')]
labels = ['Underweight', 'Normal', 'Overweight', 'Obese']
data['bmi_category'] = pd.cut(data['bmi'], bins=bins, labels=labels)
data.drop(columns=['bmi'], inplace=True)

In [ ]:
data.sample(5)

Теперь создадим таблицу сопряжённости и посчитаем хи-квадрат.

In [ ]:
# функция для создания таблицы сопряжённости
def create_contingency_table(data, col_1, col_2):
  contingency_table = pd.crosstab(data[col_1], data[col_2])

  return contingency_table


# функция для подсчёта сумм по строкам, столбцам и всей таблице
def count_totals(contingency_table):
  row_totals = contingency_table.sum(axis=1).values
  col_totals = contingency_table.sum(axis=0).values
  total = contingency_table.values.sum()

  return row_totals, col_totals, total


# функция для подсчёта хи-квадрат
def count_chi2(totals, contingency_table):
  row_totals, col_totals, total = totals
  expected_frequencies = np.outer(row_totals, col_totals) / total
  chi2_value = ((contingency_table.values - expected_frequencies) ** 2 / expected_frequencies).sum()

  return chi2_value


# функция для подсчёта p-value
def count_p_value(totals, chi2_value):
  row_totals, col_totals, _ = totals
  # степени свободы
  dof = (len(row_totals) - 1) * (len(col_totals) - 1)
  # я позволила себе шалость и взяла готовую реализацию функции распределения
  p_value = 1 - chi2.cdf(chi2_value, dof)

  return p_value

In [ ]:
contingency_table = create_contingency_table(data, 'sex', 'bmi_category')

In [ ]:
contingency_table

In [ ]:
totals = count_totals(contingency_table)
chi2_value = count_chi2(totals, contingency_table)

print(f"Chi2: {chi2_value}")

In [ ]:
p_value = count_p_value(totals, chi2_value)

print(f"p-value: {p_value}")

На основании p-value, которое больше 0.05 (сильно больше), мы не можем отклонить нулевую гипотезу (то есть не можем утверждать, что между полом и BMI есть зависимость).

На всякий случай теперь проверим наш результат без категоризации BMI.

#### Решение без категоризации BMI

In [ ]:
data = pd.read_csv('/content/sex_bmi_smokers.csv')

In [ ]:
contingency_table = create_contingency_table(data, 'sex', 'bmi')

In [ ]:
totals = count_totals(contingency_table)
chi2_value = count_chi2(totals, contingency_table)

print(f"Chi2: {chi2_value}")

In [ ]:
p_value = count_p_value(totals, chi2_value)

print(f"p-value: {p_value}")

Мы подтвердили, что не можем откинуть нулевую гипотезу.

### Точный критерий Фишера

In [ ]:
data = pd.read_csv('/content/sex_bmi_smokers.csv')

In [ ]:
data['bmi'].value_counts()

Точный критерий Фишера применим к таблицам 2$\times$2, так что нам придётся привести наши данные к такому виду (то есть категоризовать BMI, но уже на две категории - я выбрала в качестве них низкий - меньше 25 - и высокий - больше 25).

Для таблицы 2$\times$2 формула для расчёта вероятности конкретного распределения значений такая:

$$P =\frac{{(a + b)!(c + d)!(a + c)!(b + d)!}}{{a!b!c!d!n!}}$$

P-value для точного критерия Фишера рассчитывается путём суммирования вероятностей всех таких же или более экстремальных распределений таблиц, чем наблюдаемое, при условии фиксированных маргинальных сумм таблицы.

In [ ]:
bins = [0, 24.9, float('inf')]
labels = ['low', 'high']
data['bmi_category'] = pd.cut(data['bmi'], bins=bins, labels=labels)
data.drop(columns=['bmi'], inplace=True)

In [ ]:
contingency_table = create_contingency_table(data, 'sex', 'bmi_category')

In [ ]:
contingency_table_2x2 = contingency_table.loc[:, ['low', 'high']].copy()
contingency_table_2x2['high'] = contingency_table.loc[:, contingency_table.columns != 'low'].sum(axis=1)

In [ ]:
odds_ratio, p_value = fisher_exact(contingency_table_2x2)

print(f"Odds Ratio: {odds_ratio}")
print(f"p-value (Fisher's exact test): {p_value}")

P-value всё ещё довольно сильно больше 0.05. Мы снова не можем откинуть нулевую гипотезу, а значит, не можем утверждать, что есть зависимость между полом и BMI.